# Chapter 6 - Decision Tree Model

## Project: Credit Risk Score.

In [ ]:
# Importing Python Packages. 

# Data Manipulation. 
import pandas as pd 
import numpy as np

# Data Visualization Packages. 
import seaborn as sns 
import matplotlib.pyplot as plt

# Data Gathering Package. 
import wget 

# Data preparation packages.
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction import DictVectorizer

# Model Training Packages.
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import roc_auc_score

# Other
from IPython.display import display 
%matplotlib inline

## Data Gathering.

In [ ]:
# Data url link
url = 'https://github.com/gastonstat/CreditScoring/raw/master/CreditScoring.csv'

# Downloading data with wget.
# wget.download(url, 'data/credit_scoring.csv')

In [ ]:
# Loading dataset.
credit_score = pd.read_csv('data/credit_scoring.csv')

# Viewing 
credit_score.head()

## Data Cleaning & Preparation. 

In [ ]:
# Changing header to small cap.
credit_score.columns = credit_score.columns.str.replace(' ', '_').str.lower()

# Checking data columns names.
credit_score.columns

In [ ]:
credit_score.status.value_counts()

In [ ]:
# Replacing numbers with values.
status_values = {
    1: 'ok',
    2: 'default',
    0: 'unk'
}

credit_score.status = credit_score.status.map(status_values)

# Check header. 
credit_score.head()

In [ ]:
# Converting Values to str values. 
home_values = {
    1: 'rent',
    2: 'owner',
    3: 'private',
    4: 'ignorfe',
    5: 'parents',
    6: 'other',
    0: 'unk'
}

credit_score.home = credit_score.home.map(home_values)

marital_values = {
    1: 'single',
    2: 'married',
    3: 'widow',
    4: 'separated',
    5: 'divorced',
    0: 'unk'
}

credit_score.marital = credit_score.marital.map(marital_values)

records_values = {
    1: 'no',
    2: 'yes',
    0: 'unk'
}

credit_score.records = credit_score.records.map(records_values)

job_values = {
    1: 'fixed',
    2: 'partime',
    3: 'freelance',
    4: 'others',
    0: 'unk'
}

credit_score.job = credit_score.job.map(job_values)

In [ ]:
# Checking result.
credit_score.head()

In [ ]:
# Statistic overview.
credit_score.describe()

In [ ]:
# Removing Outlier with None. 

# Removing outliers for Debt column. 
credit_score.debt.replace(99999999, np.nan, inplace = True)

# Removing outlier for assets column.
credit_score.assets.replace(99999999, np.nan, inplace = True)

# Removing outlier for income column.
credit_score.income.replace(99999999, np.nan, inplace = True)

# Veiwing changes. 
credit_score[['income', 'debt', 'assets']].max()

In [ ]:
# Remove rows with unknow status.
credit_score = credit_score[credit_score.status != 'unk']

# Show result.
credit_score.status.value_counts(normalize = True)

In [ ]:
# Splitting dataset. 
# Full_Train & Test Dataset.
credit_train_full, credit_test = train_test_split(credit_score, test_size= 0.2, random_state = 10)

# Train & Validation Dataset.
credit_train, credit_val = train_test_split(credit_train_full, test_size = 0.25, random_state = 10)

# Reset index values. 
credit_train_full = credit_train_full.reset_index(drop=True)
credit_test = credit_test.reset_index(drop = True)
credit_train = credit_train.reset_index(drop = True)
credit_val = credit_val.reset_index(drop = True)

# Extract Dependent feature.
y_train = (credit_train['status']== 'ok').astype(int).values
y_test = (credit_test['status'] == 'ok').astype(int).values
y_val = (credit_val['status'] == 'ok').astype(int).values

# Delete dependent columns from dataset. 
del credit_train['status']
del credit_test['status']
del credit_val['status']

# Show all dataset. 
display(credit_train)
display(credit_test)
display(credit_val)

## Training Model: Decision Tree.

In [ ]:
# Converting data to dict.
train_dict = credit_train.fillna(0).to_dict(orient = 'records')

# Check result.
train_dict

In [ ]:
# Creating Dict Vectorizer instance.
dv = DictVectorizer(sparse = False)

# Training and transforming data
x_train = dv.fit_transform(train_dict)

# Show result 
print(x_train, '\n')

# Show feature names. 
print(dv.get_feature_names())


In [ ]:
# Training Decision Tree model. 
dtm = DecisionTreeClassifier()

# Train model with training dataset. 
dtm.fit(x_train, y_train)

In [ ]:
# Transform data to dict formate. 
val_dict = credit_val.fillna(0).to_dict(orient = 'records')

# Transform data to vectors. 
x_val = dv.transform(val_dict)

# Show result. 
x_val

In [ ]:
# Testing the model with the validation dataset.
y_predi = dtm.predict_proba(x_val)[:, 1]

# Showing result. 
y_predi

In [ ]:
# Evaluate model strenth. 
roc_score = roc_auc_score(y_predi, y_val)
roc_score

In [ ]:
# Converting data to data to dict. 
train_dict = credit_train.fillna(0).to_dict(orient = 'records')

# Converting dict data to vector. 
x_train = dv.transform(train_dict)

# Testing model with training dataset. 
y_predi = dtm.predict_proba(x_train)[:, 1]

# Evaluting model prediction. 
roc_score = roc_auc_score(y_predi, y_train)

# Show score result. 
roc_score

#### Observation: 
The roc auc score of 1 show us that our model is overfitting. 

### Using the max depth and min_sample_leaf parameter.


In [ ]:
# Creating instanse of model.
model = DecisionTreeClassifier(max_depth = 3, random_state = 10, min_samples_leaf = 30 )

# Training model with training data.
model.fit(x_train, y_train)

# Evaluate model.
y_predi = model.predict_proba(x_val)[:, 1]

# Measure effectiveness.
auc_score = roc_auc_score(y_val, y_predi)

# Show result. 
print(y_predi)
print('\n\n',f"ROC_AUC_SCORE --> {auc_score}")

In [ ]:
# Depth and Sample levels.
depth = [2, 3,  4, 6, 7, 8, 10, 15, 20, 30, 40, 50, None]

# Saved model output 
result = []

# Testing different max_depth level. 

for d in depth: 
    print('Creating instanse of Decision Tree model', '\n')
    model = DecisionTreeClassifier(max_depth = d, random_state = 10)
    
    print(f'Training Model with {d} Depth level.', '\n')
    model.fit(x_train, y_train)
    
    print('Evaluating model with validation data.', '\n')
    y_predic = model.predict_proba(x_val)[:, 1]
    
    print('Calculating Model Effectiveness', '\n')    
    auc_score = roc_auc_score(y_val, y_predic)
    
    print('Appending result ot Result-list', '\n\n')    
    result.append((d, auc_score))

In [ ]:
# Result dataframe. 
depth_auc = pd.DataFrame(result, columns = ['depth', 'auc_score'])

# Show result.
depth_auc

In [ ]:
# Testing different min_samples_leaf level. 
samples_leaf = [3, 5, 8, 10, 20, 50, 55, 57, 59, 60, 70, 80, 90, 100, 150, 200]

# Pickt the best max_depth level.
depth_level = 7

# Result. 
result = []

for s in samples_leaf: 
    print('Creating instanse of Decision Tree model', '\n')
    model = DecisionTreeClassifier(max_depth = depth_level, min_samples_leaf = s, random_state = 10)
    
    print(f'Training Model with {d} sample leaf level.', '\n')
    model.fit(x_train, y_train)
    
    print('Evaluating model with validation data.', '\n')
    y_predic = model.predict_proba(x_val)[:, 1]
    
    print('Calculating Model Effectiveness', '\n')    
    auc_score = roc_auc_score(y_val, y_predic)
    
    print('Appending result ot Result-list', '\n\n')    
    result.append((s, auc_score))

In [ ]:
# Result dataframe. 
sample_leaf_auc = pd.DataFrame(result, columns = ['samples_leaf', 'auc_score'])

# Show result.
sample_leaf_auc

In [ ]:
# Plotting Result. 
plt.plot(sample_leaf_auc.samples_leaf, sample_leaf_auc.auc_score, label = 'Min-Samples-Leaf')
plt.legend()
plt.show()

In [ ]:
# Testing different combination of verying 
# max depth and min sample leaf. 

# Saving output. 
result = []


for d in depth: 
    for s in samples_leaf: 
        print('Creating instanse of Decision Tree model', '\n')
        model = DecisionTreeClassifier(max_depth = d, min_samples_leaf = s, random_state = 10)
    
        print(f'Training Model with {d} sample leaf level.', '\n')
        model.fit(x_train, y_train)
    
        print('Evaluating model with validation data.', '\n')
        y_predic = model.predict_proba(x_val)[:, 1]
    
        print('Calculating Model Effectiveness', '\n')    
        auc_score = roc_auc_score(y_val, y_predic)
    
        print('Appending result ot Result-list', '\n\n')    
        result.append((d, s, auc_score))
        

In [ ]:
# Model result dataframe.
model_auc = pd.DataFrame(result, columns = ['max_depth', 'min_sample_leaf', 'auc_score'])

# Show result. 
model_auc.tail()

In [ ]:
# Sort on AUC score. 
model_auc.sort_values('auc_score', ascending = False).head(10)

## Training Model with Random Forest. 

In [ ]:
# Model Package.
from sklearn.ensemble import RandomForestClassifier

# Creating of model. 
rfc = RandomForestClassifier(n_estimators = 10, random_state = 15)

# Training model. 
rfc.fit(x_train, y_train)

# Evaluate model. 
y_predi = rfc.predict_proba(x_val)[:, 1]

# Check Model accuracy. 
auc_score = roc_auc_score(y_val, y_predi)

In [ ]:
# Show result. 
auc_score

In [ ]:
# Test various parameter rate. 
estimators = [x for x in range(1, 201, 10)]
depth_level = 15
min_sampls_leaf = 57
result = []
# Looping throught different rate. 
for e in estimators: 
    print('Creating instanse of Decision Tree model', '\n')
    model = RandomForestClassifier(n_estimators = e, max_depth = depth_level, min_samples_leaf = s, random_state = 15)
    
    print(f'Training Model with {d} sample leaf level.', '\n')
    model.fit(x_train, y_train)
    
    print('Evaluating model with validation data.', '\n')
    y_predic = model.predict_proba(x_val)[:, 1]
    
    print('Calculating Model Effectiveness', '\n')    
    auc_score = roc_auc_score(y_val, y_predic)
    
    print('Appending result ot Result-list', '\n\n')    
    result.append((e, auc_score))

In [ ]:
# Output dataframe. 
result = pd.DataFrame(result, columns= ['estimator', 'auc_score'])

# Check table. 
result

In [ ]:
result.sort_values(by = 'auc_score', ascending = False)

In [ ]:
# Plotting model accuracy with various estimator rate. 
plt.plot(result.estimator, result.auc_score)
plt.show()

In [ ]:
# Testing Various level of max_dept, min_sample_leaf, 
# n_estimators, max_feature, 
result = []
depth = [2, 3,  4, 6, 7, 8, 10, 15, 20, 30, 40, 50, None]
estimators = [x for x in range(1, 201, 10)]
samples_leaf = [3, 5, 8, 10, 20, 50, 55, 57, 59, 60, 70, 80, 90, 100, 150, 200]
n_feature = [0.2, 0.5, 0.4, 0.7, 0.8, 1.0]

for d in depth: 
    for e in estimators:
        for s in samples_leaf: 
            for n in n_feature:
                print('Creating instanse of Decision Tree model', '\n')
                model = RandomForestClassifier(n_estimators = e, max_depth = d, min_samples_leaf = s,
                                               max_features = n, random_state = 15)
    
                print(f'Training Model with {d} sample leaf level.', '\n')
                model.fit(x_train, y_train)
    
                print('Evaluating model with validation data.', '\n')
                y_predic = model.predict_proba(x_val)[:, 1]
                
                print('Calculating Model Effectiveness', '\n')    
                auc_score = roc_auc_score(y_val, y_predic)
    
                print('Appending result ot Result-list', '\n\n')    
                result.append(( d, e, s, n, auc_score))
                


Creating instanse of Decision Tree model 

Training Model with 2 sample leaf level. 

Evaluating model with validation data. 

Calculating Model Effectiveness 

Appending result ot Result-list 


Creating instanse of Decision Tree model 

Training Model with 2 sample leaf level. 

Evaluating model with validation data. 

Calculating Model Effectiveness 

Appending result ot Result-list 


Creating instanse of Decision Tree model 

Training Model with 2 sample leaf level. 

Evaluating model with validation data. 

Calculating Model Effectiveness 

Appending result ot Result-list 


Creating instanse of Decision Tree model 

Training Model with 2 sample leaf level. 

Evaluating model with validation data. 

Calculating Model Effectiveness 

Appending result ot Result-list 


Creating instanse of Decision Tree model 

Training Model with 2 sample leaf level. 

Evaluating model with validation data. 

Calculating Model Effectiveness 

Appending result ot Result-list 


Creating instanse of

Evaluating model with validation data. 

Calculating Model Effectiveness 

Appending result ot Result-list 


Creating instanse of Decision Tree model 

Training Model with 2 sample leaf level. 

Evaluating model with validation data. 

Calculating Model Effectiveness 

Appending result ot Result-list 


Creating instanse of Decision Tree model 

Training Model with 2 sample leaf level. 

Evaluating model with validation data. 

Calculating Model Effectiveness 

Appending result ot Result-list 


Creating instanse of Decision Tree model 

Training Model with 2 sample leaf level. 

Evaluating model with validation data. 

Calculating Model Effectiveness 

Appending result ot Result-list 


Creating instanse of Decision Tree model 

Training Model with 2 sample leaf level. 

Evaluating model with validation data. 

Calculating Model Effectiveness 

Appending result ot Result-list 


Creating instanse of Decision Tree model 

Training Model with 2 sample leaf level. 

Evaluating model wit

Evaluating model with validation data. 

Calculating Model Effectiveness 

Appending result ot Result-list 


Creating instanse of Decision Tree model 

Training Model with 2 sample leaf level. 

Evaluating model with validation data. 

Calculating Model Effectiveness 

Appending result ot Result-list 


Creating instanse of Decision Tree model 

Training Model with 2 sample leaf level. 

Evaluating model with validation data. 

Calculating Model Effectiveness 

Appending result ot Result-list 


Creating instanse of Decision Tree model 

Training Model with 2 sample leaf level. 

Evaluating model with validation data. 

Calculating Model Effectiveness 

Appending result ot Result-list 


Creating instanse of Decision Tree model 

Training Model with 2 sample leaf level. 

Evaluating model with validation data. 

Calculating Model Effectiveness 

Appending result ot Result-list 


Creating instanse of Decision Tree model 

Training Model with 2 sample leaf level. 

Evaluating model wit

Evaluating model with validation data. 

Calculating Model Effectiveness 

Appending result ot Result-list 


Creating instanse of Decision Tree model 

Training Model with 2 sample leaf level. 

Evaluating model with validation data. 

Calculating Model Effectiveness 

Appending result ot Result-list 


Creating instanse of Decision Tree model 

Training Model with 2 sample leaf level. 

Evaluating model with validation data. 

Calculating Model Effectiveness 

Appending result ot Result-list 


Creating instanse of Decision Tree model 

Training Model with 2 sample leaf level. 

Evaluating model with validation data. 

Calculating Model Effectiveness 

Appending result ot Result-list 


Creating instanse of Decision Tree model 

Training Model with 2 sample leaf level. 

Evaluating model with validation data. 

Calculating Model Effectiveness 

Appending result ot Result-list 


Creating instanse of Decision Tree model 

Training Model with 2 sample leaf level. 

Evaluating model wit

Evaluating model with validation data. 

Calculating Model Effectiveness 

Appending result ot Result-list 


Creating instanse of Decision Tree model 

Training Model with 2 sample leaf level. 

Evaluating model with validation data. 

Calculating Model Effectiveness 

Appending result ot Result-list 


Creating instanse of Decision Tree model 

Training Model with 2 sample leaf level. 

Evaluating model with validation data. 

Calculating Model Effectiveness 

Appending result ot Result-list 


Creating instanse of Decision Tree model 

Training Model with 2 sample leaf level. 

Evaluating model with validation data. 

Calculating Model Effectiveness 

Appending result ot Result-list 


Creating instanse of Decision Tree model 

Training Model with 2 sample leaf level. 

Evaluating model with validation data. 

Calculating Model Effectiveness 

Appending result ot Result-list 


Creating instanse of Decision Tree model 

Training Model with 2 sample leaf level. 

Evaluating model wit

...

Evaluating model with validation data. 

Calculating Model Effectiveness 

Appending result ot Result-list 


Creating instanse of Decision Tree model 

Training Model with 4 sample leaf level. 

Evaluating model with validation data. 

Calculating Model Effectiveness 

Appending result ot Result-list 


Creating instanse of Decision Tree model 

Training Model with 4 sample leaf level. 

Evaluating model with validation data. 

Calculating Model Effectiveness 

Appending result ot Result-list 


Creating instanse of Decision Tree model 

Training Model with 4 sample leaf level. 

Evaluating model with validation data. 

Calculating Model Effectiveness 

Appending result ot Result-list 


Creating instanse of Decision Tree model 

Training Model with 4 sample leaf level. 

Evaluating model with validation data. 

Calculating Model Effectiveness 

Appending result ot Result-list 


Creating instanse of Decision Tree model 

Training Model with 4 sample leaf level. 

Evaluating model wit

Evaluating model with validation data. 

Calculating Model Effectiveness 

Appending result ot Result-list 


Creating instanse of Decision Tree model 

Training Model with 4 sample leaf level. 

Evaluating model with validation data. 

Calculating Model Effectiveness 

Appending result ot Result-list 


Creating instanse of Decision Tree model 

Training Model with 4 sample leaf level. 

Evaluating model with validation data. 

Calculating Model Effectiveness 

Appending result ot Result-list 


Creating instanse of Decision Tree model 

Training Model with 4 sample leaf level. 

Evaluating model with validation data. 

Calculating Model Effectiveness 

Appending result ot Result-list 


Creating instanse of Decision Tree model 

Training Model with 4 sample leaf level. 

Evaluating model with validation data. 

Calculating Model Effectiveness 

Appending result ot Result-list 


Creating instanse of Decision Tree model 

Training Model with 4 sample leaf level. 

Evaluating model wit

Evaluating model with validation data. 

Calculating Model Effectiveness 

Appending result ot Result-list 


Creating instanse of Decision Tree model 

Training Model with 4 sample leaf level. 

Evaluating model with validation data. 

Calculating Model Effectiveness 

Appending result ot Result-list 


Creating instanse of Decision Tree model 

Training Model with 4 sample leaf level. 

Evaluating model with validation data. 

Calculating Model Effectiveness 

Appending result ot Result-list 


Creating instanse of Decision Tree model 

Training Model with 4 sample leaf level. 

Evaluating model with validation data. 

Calculating Model Effectiveness 

Appending result ot Result-list 


Creating instanse of Decision Tree model 

Training Model with 4 sample leaf level. 

Evaluating model with validation data. 

Calculating Model Effectiveness 

Appending result ot Result-list 


Creating instanse of Decision Tree model 

Training Model with 4 sample leaf level. 

Evaluating model wit

Evaluating model with validation data. 

Calculating Model Effectiveness 

Appending result ot Result-list 


Creating instanse of Decision Tree model 

Training Model with 4 sample leaf level. 

Evaluating model with validation data. 

Calculating Model Effectiveness 

Appending result ot Result-list 


Creating instanse of Decision Tree model 

Training Model with 4 sample leaf level. 

Evaluating model with validation data. 

Calculating Model Effectiveness 

Appending result ot Result-list 


Creating instanse of Decision Tree model 

Training Model with 4 sample leaf level. 

Evaluating model with validation data. 

Calculating Model Effectiveness 

Appending result ot Result-list 


Creating instanse of Decision Tree model 

Training Model with 4 sample leaf level. 

Evaluating model with validation data. 

Calculating Model Effectiveness 

Appending result ot Result-list 


Creating instanse of Decision Tree model 

Training Model with 4 sample leaf level. 

Evaluating model wit

Evaluating model with validation data. 

Calculating Model Effectiveness 

Appending result ot Result-list 


Creating instanse of Decision Tree model 

Training Model with 4 sample leaf level. 

Evaluating model with validation data. 

Calculating Model Effectiveness 

Appending result ot Result-list 


Creating instanse of Decision Tree model 

Training Model with 4 sample leaf level. 

Evaluating model with validation data. 

Calculating Model Effectiveness 

Appending result ot Result-list 


Creating instanse of Decision Tree model 

Training Model with 4 sample leaf level. 

Evaluating model with validation data. 

Calculating Model Effectiveness 

Appending result ot Result-list 


Creating instanse of Decision Tree model 

Training Model with 4 sample leaf level. 

Evaluating model with validation data. 

Calculating Model Effectiveness 

Appending result ot Result-list 


Creating instanse of Decision Tree model 

Training Model with 4 sample leaf level. 

Evaluating model wit

In [38]:
# Create a dataframe. 
output = pd.DataFrame(result, columns = ['max_depth', 'num_estimators', 'min_sample_leaf', 'max_feature', 'auc_score'])

# Checking output. 
output

,max_depth,num_estimators,min_sample_leaf,max_feature,auc_score
0,2,1,3,0.2,0.627566
1,2,1,3,0.5,0.683265
2,2,1,3,0.4,0.679126
3,2,1,3,0.7,0.650380
4,2,1,3,0.8,0.679126
...,...,...,...,...,...
1891,2,191,80,0.5,0.779517
1892,2,191,80,0.4,0.781007
1893,2,191,80,0.7,0.775853
1894,2,191,80,0.8,0.777423


In [40]:
output.sort_values(by='auc_score', ascending = False )

,max_depth,num_estimators,min_sample_leaf,max_feature,auc_score
948,2,91,150,0.2,0.802560
1050,2,101,200,0.2,0.802080
1038,2,101,100,0.2,0.801597
1614,2,161,100,0.2,0.801383
942,2,91,100,0.2,0.801362
...,...,...,...,...,...
24,2,1,20,0.2,0.627566
12,2,1,8,0.2,0.627566
6,2,1,5,0.2,0.627566
0,2,1,3,0.2,0.627566


## Using XGBoost to train model. 

In [ ]:
# Import xgboost package as xgb.
import xgboost as xgb

# Feature name. 
feature = dv.get_feature_name()

# Training Dmetric. 
dtrain = xgb.DMatric(x_train, label = y_train, feature_name = feature)

# validation 

In [ ]:

# Craete instance of model.
xgb_param = {
    'etc': 0.3
    'max_depth': 6, 
    'min_child_weight': 150, 
    
    
    
}

model_xgb = xgb(xgb_param, )